In [1]:
from pyxirr import xirr
import streamlit as st
import pandas as pd
import yfinance as yf
import numpy as np
import pandas_datareader as reader
#from scipy.stats import norm
from datetime import datetime
from datetime import timedelta
from datetime import date
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import matplotlib
import investpy as inv
plt.style.use('default')
import warnings 
warnings.filterwarnings(action='ignore')
import plotly.express as px
import plotly.graph_objects as go
import quantstats as qs
from matplotlib import colors
from scipy import stats

In [2]:

movimientos = pd.read_excel("records.xlsx", parse_dates=True)
movimientos = movimientos.set_index("Fecha")

opcion_seleccionada = "Omite monetarios"

if opcion_seleccionada == "Omite monetarios":
    movimientos = movimientos[(movimientos["Description"] != "ETF monetario") & (movimientos["Description"] != "Fondo monetario")]
    fecha_inicio = "2023-09-01"
else:
    fecha_inicio = "2023-01-01"

fecha_hoy = datetime.now()
fecha_formateada = fecha_hoy.strftime("%Y-%m-%d")

rango_fechas = pd.date_range(fecha_inicio, end=fecha_formateada, freq="D")
rango_fechas = rango_fechas[rango_fechas.dayofweek < 5]

# Download prices
precios = pd.DataFrame(index=rango_fechas)
for i in movimientos["Yahoo Ticker"].dropna().unique():
    precios[i] = yf.download(i, start=fecha_inicio, progress=False)["Adj Close"]

eurusd = yf.download("EURUSD=X", start=fecha_inicio, progress=False).resample("B").ffill()["Adj Close"]

precios["WBIT"] = yf.download("BTC-USD", start=fecha_inicio, progress=False).resample("B").ffill()["Adj Close"] / eurusd * 0.0002396
for ticker in ["JOE", "BN", "BAM"]:
    precios[ticker] = precios[ticker] / eurusd

precios = precios.fillna(method="ffill")

# Rename columns
column_mapping_prices = {
    'CSH2.PA': '0.0 ETF monetario', '0P00002BDB.F': '0.1 Fondo monetario', 'U3O8.DE': '1.6 Uranio',
    'ZPRV.DE': '1.3 USA Small Value', '0P0001AINF.F': '1.1 World', '0P0001AINL.F': '1.4 Emergentes',
    'SMCX.MI': '1.2 Europa Small', 'BN': '2.2 Brookfield Corp', 'JOE': '2.3 St Joe', 'TL0.DE': '2.1 Tesla',
    'WBIT': '1.5 ETF bitcoin', 'BAM': '2.4 Brookfield AM'
}

column_mapping_else = {'ETF monetario': '0.0 ETF monetario', 'Fondo monetario': '0.1 Fondo monetario', 'Uranio': '1.6 Uranio', 'USA Small Value': '1.3 USA Small Value', 'World': '1.1 World',
'Emergentes': '1.4 Emergentes', 'Europa Small': '1.2 Europa Small', 'Brookfield Corp': '2.2 Brookfield Corp', 
'St Joe': '2.3 St Joe', 'Tesla': '2.1 Tesla', 'ETF bitcoin': '1.5 ETF bitcoin', 'Brookfield AM': '2.4 Brookfield AM'
                    }

precios.rename(columns=column_mapping_prices, inplace=True)
precios = precios.sort_index(axis=1)

# Calculate returns
rendimientos = precios.pct_change()

# Download and calculate benchmark returns
benchmark = yf.download("SPYI.DE", start=fecha_inicio, progress=False).resample("B").ffill()["Adj Close"]
rendimiento_benchmark = benchmark.pct_change().fillna(0)

# Calculate positions
posiciones = pd.DataFrame(index=rango_fechas, columns=movimientos["Description"].unique())
for i in movimientos["Description"].unique():
    posiciones[i] = movimientos[movimientos["Description"] == i].cumsum()["Flow unidades"]
posiciones = posiciones.fillna(method="ffill").fillna(0)
posiciones.rename(columns=column_mapping_else, inplace=True)
posiciones = posiciones.sort_index(axis=1)

# Calculate cost
coste = pd.DataFrame(index=rango_fechas, columns=movimientos["Description"].unique())
for i in movimientos["Description"].unique():
    coste[i] = movimientos[movimientos["Description"] == i].cumsum()["Flow"]
coste = coste.fillna(method="ffill").fillna(0)
coste.rename(columns=column_mapping_else, inplace=True)
coste = coste[posiciones.columns]

# Calculate value and weights
valor = precios * posiciones
pesos = valor.divide(valor.sum(axis=1), axis=0)

# Calculate contribution and portfolio return
contribucion = pesos.shift() * rendimientos
rendimiento_portfolio = contribucion.sum(axis=1)

# Calculate P&L
pl = valor.add(coste)

# Update movimientos
movimientos.replace(column_mapping_else, inplace=True)
movimientos["Ud sim benchmark"] = (-movimientos["Flow"] / benchmark.loc[movimientos.index]).cumsum()





In [79]:
############################ intra dia

import pandas as pd
import yfinance as yf
from datetime import datetime
import pytz

# Set time zone to Madrid
madrid_tz = pytz.timezone("Europe/Madrid")

ny_tz = pytz.timezone("America/New_York")

ccy_tz = pytz.timezone("Etc/GMT-1")

btc_tz = pytz.timezone("Etc/UTC")

# Define date range
fecha_inicio = datetime.now() - timedelta(4) 
fecha_hoy = datetime.now(madrid_tz)  # Ensure current date is in Madrid timezone
fecha_formateada = fecha_hoy.strftime("%Y-%m-%d %H:%M:%S")

# Generate a range of 15-minute intervals from 08:00 to 23:00, Monday to Sunday
rango_fechas = pd.date_range(start=fecha_inicio, end=fecha_formateada, freq='15T', tz=madrid_tz)

# Filter time range to keep only between 08:00 and 23:00
rango_fechas = rango_fechas[(rango_fechas.hour >= 8) & (rango_fechas.hour < 23)]

# Create an empty DataFrame for prices
precios = pd.DataFrame(index=rango_fechas)

# Crear una copia de los tickers originales
tickers = movimientos["Yahoo Ticker"].dropna().unique().copy()

# Reemplazar '0P0001AINL.F' por 'IWDA.AS' y '0P0001AINF.F' por 'EUNM.DE' en la copia
tickers = [ticker.replace('0P0001AINL.F', 'IWDA.AS').replace('0P0001AINF.F', 'EUNM.DE') for ticker in tickers]


# Ahora puedes usar la lista `tickers` para el bucle sin afectar al DataFrame original


# Download intraday data for each ticker and resample it for 15-min intervals
for i in tickers:


    if i in ['U3O8.DE', 'ZPRV.DE', 'IWDA.AS', 'EUNM.DE', 'SMCX.MI','TL0.DE']:

        data = yf.download(i, start=fecha_inicio, interval="15m", progress=False)
        data = data.tz_localize(madrid_tz)  # Convert data to Madrid timezone
        precios[i] = data["Adj Close"].reindex(rango_fechas, method="ffill")  # Resample and forward-fill missing data

    if i in ['BAM', 'BN', 'JOE']:

        data = yf.download(i, start=fecha_inicio, interval="15m", progress=False)
        data = data.tz_localize(ny_tz) 
        precios[i] = data["Adj Close"].reindex(rango_fechas, method="ffill")  # Resample and forward-fill missing data # Convert data to Madrid timezone

# Download EUR/USD exchange rate data (15-min intervals)

eurusd = yf.download("EURUSD=X", start=fecha_inicio, interval="15m", progress=False)
eurusd = eurusd.tz_localize(ccy_tz).reindex(rango_fechas, method="ffill")["Adj Close"]

# Download Bitcoin data (BTC-USD) and adjust for EUR/USD conversion

btc_usd = yf.download("BTC-USD", start=fecha_inicio, interval="15m", progress=False)
btc_usd = btc_usd.tz_localize(btc_tz).reindex(rango_fechas, method="ffill")["Adj Close"]
precios["WBIT"] = btc_usd / eurusd * 0.0002396

# Adjust other tickers for EUR/USD
for ticker in ["JOE", "BN", "BAM"]:
   precios[ticker] = precios[ticker] / eurusd

# Fill forward missing data
precios = precios.fillna(method="ffill")

# Optionally, filter out weekends if desired (optional)
# precios = precios[precios.index.dayofweek < 5]

# Display the final DataFrame


precios


,U3O8.DE,ZPRV.DE,IWDA.AS,EUNM.DE,SMCX.MI,BAM,BN,JOE,TL0.DE,WBIT
2024-10-10 20:38:20.727288+02:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-10-10 20:53:20.727288+02:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-10-10 21:08:20.727288+02:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-10-10 21:23:20.727288+02:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-10-10 21:38:20.727288+02:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2024-10-14 19:23:20.727288+02:00,9.593,63.27,99.574997,38.618999,314.950012,45.002737,49.433293,54.505961,200.25,14.401785
2024-10-14 19:38:20.727288+02:00,9.593,63.27,99.574997,38.618999,314.950012,44.960510,49.389622,54.410195,200.25,14.444386
2024-10-14 19:53:20.727288+02:00,9.593,63.27,99.574997,38.618999,314.950012,44.996545,49.392599,54.132453,200.25,14.426040
2024-10-14 20:08:20.727288+02:00,9.593,63.27,99.574997,38.618999,314.950012,44.987375,49.351345,54.137038,200.25,14.461121


In [4]:
# Remove days with zero returns
returns_nonzero = rendimiento_portfolio[rendimiento_portfolio != 0]

# Calculate statistics
mean_return = returns_nonzero.mean()
median_return = returns_nonzero.median()
std_dev = returns_nonzero.std()

# Calculate the range for the bins
max_abs_return = max(abs(returns_nonzero.min()), abs(returns_nonzero.max()))
min_return = -np.ceil(max_abs_return * 1000) / 1000
max_return = np.ceil(max_abs_return * 1000) / 1000

# Create bins for every 10 basis points
bins = np.arange(min_return, max_return + 0.001, 0.001)

# Create the Plotly figure
fig = go.Figure()

# Add histogram trace
fig.add_trace(go.Histogram(
    x=returns_nonzero,
    name='Daily Returns',
    opacity=0.75,
    #marker_color='blue',
    xbins=dict(
        start=min_return,
        end=max_return,
        size=0.001
    ),
    autobinx=False
))

# Add density curve
kde = stats.gaussian_kde(returns_nonzero)
x_range = np.linspace(min_return, max_return, 1000)
y_range = kde(x_range)
fig.add_trace(go.Scatter(x=x_range, y=y_range * len(returns_nonzero) * 0.001, 
                         mode='lines', name='Density', line=dict(color='darkorange')))

# Update layout
fig.update_layout(
    title='Histogram of Portfolio Daily Returns',
    xaxis_title='Return',
    yaxis_title='Frequency',
    bargap=0.05,  # Gap between bars
    showlegend=False,
    xaxis=dict(
        tickformat='.1%',  # Format x-axis labels as percentages
        tickmode='array',
        tickvals=np.arange(min_return, max_return + 0.001, 0.01),  # Major ticks every 1%
        ticktext=[f'{x:.1%}' for x in np.arange(min_return, max_return + 0.001, 0.01)],
        range=[min_return, max_return]  # Center the graph
    )
)

# Add a vertical line at x=0
fig.add_vline(x=0, line_dash="dash", line_color="white")

# Add statistics box
stats_text = (f"Mean: {mean_return:.2%}<br>"
              f"Median: {median_return:.2%}<br>"
              f"Std Dev: {std_dev:.2%}")

fig.add_annotation(
    xref="paper", yref="paper",
    x=0.95, y=0.95,
    text=stats_text,
    showarrow=False,
    font=dict(size=12),
    align="left",
    #bgcolor="white",
    #bordercolor="black",
    borderwidth=1
)

# Show the plot
fig

In [5]:
import datetime

# Remove days with zero returns
returns_nonzero = rendimiento_portfolio[rendimiento_portfolio != 0]

# Calculate statistics
mean_return = returns_nonzero.mean()
median_return = returns_nonzero.median()
std_dev = returns_nonzero.std()

# Calculate the range for the bins
max_abs_return = max(abs(returns_nonzero.min()), abs(returns_nonzero.max()))
min_return = -np.ceil(max_abs_return * 1000) / 1000
max_return = np.ceil(max_abs_return * 1000) / 1000

# Create bins for every 10 basis points
bins = np.arange(min_return, max_return + 0.001, 0.001)

# Get the returns from the last 10 days and their dates
last_10_returns = returns_nonzero[-10:]
last_10_dates = rendimiento_portfolio.index[-10:]

# Highlight the bins that contain the last 10 days' returns
highlight_bins = np.digitize(last_10_returns, bins)

# Calculate the histogram manually to get bin heights
hist_values, hist_bins = np.histogram(returns_nonzero, bins=bins)

# Create the Plotly figure
fig = go.Figure()

# Add histogram trace
fig.add_trace(go.Histogram(
    x=returns_nonzero,
    name='Daily Returns',
    opacity=0.75,
    xbins=dict(
        start=min_return,
        end=max_return,
        size=0.001
    ),
    autobinx=False,
    marker_color='blue'  # Default bin color, no additional formatting
))

# Highlight bins for the last 10 days with yellow color
for bin_idx in highlight_bins:
    bin_value = bins[bin_idx - 1]  # Get the left edge of the bin
    fig.add_shape(type="rect",
                  x0=bin_value, x1=bin_value + 0.001,  # Bin size is 0.001
                  y0=0, y1=hist_values[bin_idx - 1],  # Respect original bin height
                  fillcolor="yellow", opacity=0.5, line_width=0)

# Add density curve
kde = stats.gaussian_kde(returns_nonzero)
x_range = np.linspace(min_return, max_return, 1000)
y_range = kde(x_range)
fig.add_trace(go.Scatter(x=x_range, y=y_range * len(returns_nonzero) * 0.001, 
                         mode='lines', name='Density', line=dict(color='darkorange')))

# Update layout
fig.update_layout(
    title='Histogram of Portfolio Daily Returns',
    xaxis_title='Return',
    yaxis_title='Frequency',
    bargap=0.05,  # Gap between bars
    showlegend=False,
    xaxis=dict(
        tickformat='.1%',  # Format x-axis labels as percentages
        tickmode='array',
        tickvals=np.arange(min_return, max_return + 0.001, 0.01),  # Major ticks every 1%
        ticktext=[f'{x:.1%}' for x in np.arange(min_return, max_return + 0.001, 0.01)],
        range=[min_return, max_return]  # Center the graph
    )
)

# Add a vertical line at x=0
fig.add_vline(x=0, line_dash="dash", line_color="white")

# Add statistics box
stats_text = (f"Mean: {mean_return:.2%}<br>"
              f"Median: {median_return:.2%}<br>"
              f"Std Dev: {std_dev:.2%}")

fig.add_annotation(
    xref="paper", yref="paper",
    x=0.95, y=0.95,
    text=stats_text,
    showarrow=False,
    font=dict(size=12),
    align="left",
    borderwidth=1
)

# Add a text box with the last 10 days' returns formatted with dates
last_10_text = "<br>".join([f"{datetime.datetime.strftime(date, '%b %d')}: {rtn:.2%}" 
                            for date, rtn in zip(last_10_dates, last_10_returns)])
fig.add_annotation(
    xref="paper", yref="paper",
    x=0.95, y=0.75,  # Position under the stats box
    text=f"Last 10 Days:<br>{last_10_text}",
    showarrow=False,
    font=dict(size=10),
    align="left",
    borderwidth=1
)

# Show the plot
fig
